In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns


In [ ]:
SIZE = 128

train_images = []
train_labels = [] 
for directory_path in glob.glob("../input/hotels10/Train/*"):
    label = directory_path.split("\\")[-1]
    print(label)#se imprime la clase
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)#imagen       
        img = cv2.resize(img, (SIZE, SIZE))#ajuste de dimensiones a 128
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)#en RGB
        train_images.append(img)
        train_labels.append(label)        

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
# test
test_images = []
test_labels = [] 
for directory_path in glob.glob("../input/hotels10/Test/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)
        
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
print(train_labels)

In [ ]:
#Sustituir nombres de clases por numeros
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
print(test_labels_encoded)

In [ ]:
#dividir datos
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
print(y_train)

In [ ]:
# Normalizar valores de pixeles
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
#One hot encode
from tensorflow.keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
print(y_test_one_hot)

In [ ]:
# Configuracion de la red para la extraccion de caracteristicas
activation = 'relu'

feature_extractor = Sequential()
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (SIZE, SIZE, 3)))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Flatten())

In [ ]:
import tensorflow as tf
#Se suma la capa densa y de clasificacion a la extraccion de caracteristicas 
x = feature_extractor.output  
x = Dense(128, activation = activation, kernel_initializer = 'he_uniform')(x)
#Capa de clasificacion
prediction_layer = Dense(4, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation = 'softmax')(x)#SVM
#prediction_layer = Dense(4, activation = 'softmax')(x) #NO SVM

In [ ]:
# Modelo con la extraccion de caracteristicas y la clasificacion
cnn_model = Model(inputs=feature_extractor.input, outputs=prediction_layer)
cnn_model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(cnn_model.summary()) 

In [ ]:
#Entrenamiento
history = cnn_model.fit(x_train, y_train_one_hot, epochs=30, validation_data = (x_test, y_test_one_hot))

In [ ]:
#Graficas
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#Predicciones
prediction_NN = cnn_model.predict(x_test)

In [ ]:
print(prediction_NN)

In [ ]:
prediction_NN = np.argmax(prediction_NN, axis=-1)

In [ ]:
print(prediction_NN)

In [ ]:
prediction_NN = le.inverse_transform(prediction_NN)

In [ ]:
#Evaluacion
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, prediction_NN)
print(cm)
sns.heatmap(cm, annot=True)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, prediction_NN))

In [ ]:
#Ejemplo de clasificacion
n=12  
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
prediction = np.argmax(cnn_model.predict(input_img))  #argmax to convert categorical back to original
prediction = le.inverse_transform([prediction])  #Reverse the label encoder to original name
print("The prediction for this image is: ", prediction)
print("The actual label for this image is: ", test_labels[n])

In [ ]:
#Extraccion de caracteristicas con RF
X_for_RF = feature_extractor.predict(x_train) #Extraccion de caracteristicas

#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

# Entrenar el modelo
RF_model.fit(X_for_RF, y_train) 

#Extraccion de caracteristicas de test
X_test_feature = feature_extractor.predict(x_test)
#Prediccion
prediction_RF = RF_model.predict(X_test_feature)
#Invertir label al original
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
#Matriz de confusion
cm = confusion_matrix(test_labels, prediction_RF)
print(cm)
sns.heatmap(cm, annot=True)

In [ ]:
#Clasificacion de ejemplo
n=10  
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) 
prediction = np.argmax(cnn_model.predict(input_img))
prediction = le.inverse_transform([prediction])  
print("The prediction for this image is: ", prediction)
print("The actual label for this image is: ", test_labels[n])

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16

**VGG16**

In [ ]:
SIZE = 256  #Dimension

#Labels
train_images = []
train_labels = [] 

for directory_path in glob.glob("/content/Train/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

#lists to arrays        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
test_images = []
test_labels = [] 
for directory_path in glob.glob("/content/Test/*"):
    fruit_label = directory_path.split("\\")[-1]
    print(fruit_label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)

#Convert lists to arrays                
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
# Normalizar
x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encode 
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
#Modelo sin clasificacion
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

#Capas no entrenables
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  

In [ ]:
#Extraccion de caracteristicas con vgg16 y clasificacion RF
feature_extractor=VGG_model.predict(x_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_RF = features 

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

# Entrenar al modelo
RF_model.fit(X_for_RF, y_train) 

#Extraccion de caracteristicas de tes
X_test_feature = VGG_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
#Prediccion
prediction_RF = RF_model.predict(X_test_features)
#Label original
prediction_RF = le.inverse_transform(prediction_RF)


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, prediction_RF)
print(cm)
sns.heatmap(cm, annot=True)

In [ ]:
#Ejemplo de clasificacion con VGG16 y RF
n=np.random.randint(0, x_test.shape[0])
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img_feature=VGG_model.predict(input_img)
input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
prediction_RF = RF_model.predict(input_img_features)[0] 
prediction_RF = le.inverse_transform([prediction_RF])  #Reverse the label encoder to original name
print("The prediction for this image is: ", prediction_RF)
print("The actual label for this image is: ", test_labels[n])